In [1]:
import os
import ipywidgets as widgets
from IPython.display import display

# PanCan TRIBE2 analysis

## Preliminary work
A baseline for pathway mutation averages needed to be established. In order to do this, a parser for files shipped with [PathwayMapper](http://www.pathwaymapper.org/) was hand-rolled along with a custom data structure describing the pathway contents and hierarchy. Here we show an example of a parsed pathway.

In [2]:
import pathways as lpw

@widgets.interact(pathway=[filename for filename in os.listdir('./pathways')])
def show_pathway(pathway):
    pw = lpw.parse_pathway('./pathways/' + pathway)
    print(f"Name: {pw[0]}, Contents:\n{pw[1]}")

interactive(children=(Dropdown(description='pathway', options=('TGF-Beta.txt', 'HIPPO.txt', 'WNT.txt', 'NRF2.t…

The average mutation is calculated for any given pathway and patient by only considering pathogenic mutations and the maximum mutation percentage detected. Complexes or families count as a single gene towards the overall average mutation (no weights applied).

The average mutation on all pathways for a random patient follows:

In [3]:
import pandas
from analysis import calculate_patient_mutations

pathways = []
for pw in os.listdir('./pathways'):
    pathway = lpw.parse_pathway('./pathways/' + pw)
    pathways.append(pathway)

pathways.sort(key=lambda x: x[0])
    
patients_log = pandas.read_csv('TRIBE2_db.csv')
mutations_data = pandas.read_csv('TRIBE2_seq_res.csv')

result = calculate_patient_mutations('CB224', mutations_data, pathways)
print(result)

{'Cell Cycle': 2.5, 'HIPPO': 0.0, 'MYC': 0.0, 'NOTCH': 0.0, 'NRF2': 0.0, 'PI3K': 0.0, 'RTK-RAS': 3.4166666666666665, 'TGF-Beta': 0.0, 'TP53': 4.166666666666667, 'WNT': 2.3076923076923075}


Patients have been split into two groups, according to the treatment they had (arm0, arm1). This will hold for all future analysis.
Here we show statistics about mutations for the two groups of patients and how the mutations for each pathway correlate with **dpfs**

In [4]:
from analysis import process_patients

arm0_df = process_patients(patients_log[patients_log['arm'] == 0]['PatientFirstName'])
arm0_df.describe()

,Cell Cycle,HIPPO,MYC,NOTCH,NRF2,PI3K,RTK-RAS,TGF-Beta,TP53,WNT
count,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000
mean,4.721605,0.025926,0.746914,1.491770,0.415638,2.170595,4.190329,2.254321,7.045267,3.664292
std,3.045467,0.193547,3.021788,2.307105,2.100991,3.117726,3.304840,4.078182,4.383254,2.437574
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.250000,0.000000,4.500000,2.153846
50%,4.800000,0.000000,0.000000,0.000000,0.000000,0.000000,3.583333,0.000000,7.583333,3.346154
75%,6.675000,0.000000,0.000000,2.833333,0.000000,4.136364,5.541667,3.950000,10.333333,5.076923
max,18.100000,1.700000,17.500000,11.600000,14.666667,20.727273,15.833333,16.600000,19.666667,11.923077


In [5]:
pandas.set_option("display.precision", 11)
arm0_db_df = arm0_df.join(patients_log.set_index('PatientFirstName'), on='PatientFirstName')
print(f"{arm0_db_df[['dpfs'] + [pw[0] for pw in pathways]].corr().iloc[0]}")

dpfs          1.00000000000
Cell Cycle   -0.04440247702
HIPPO         0.04245896105
MYC           0.26819358259
NOTCH         0.05562896088
NRF2          0.20949607706
PI3K          0.16945628397
RTK-RAS      -0.08269877738
TGF-Beta     -0.04376672946
TP53         -0.01443085081
WNT           0.12680323858
Name: dpfs, dtype: float64


In [6]:
arm1_df = process_patients(patients_log[patients_log['arm'] == 1]['PatientFirstName'])
arm1_df.describe()

,Cell Cycle,HIPPO,MYC,NOTCH,NRF2,PI3K,RTK-RAS,TGF-Beta,TP53,WNT
count,134.00000000000,134.00000000000,134.00000000000,134.00000000000,134.00000000000,134.00000000000,134.00000000000,134.00000000000,134.00000000000,134.00000000000
mean,4.24925373134,0.03134328358,0.91044776119,1.27711442786,0.48258706468,1.99728629579,4.86940298507,1.56567164179,6.53980099502,3.51894374282
std,2.93671972567,0.26961903238,3.08988207325,2.04632816182,2.54009359321,2.94420748388,3.69884275634,3.65780871330,4.77239604779,2.19208570035
min,0.00000000000,0.00000000000,0.00000000000,0.00000000000,0.00000000000,0.00000000000,0.00000000000,0.00000000000,0.00000000000,0.00000000000
25%,2.30000000000,0.00000000000,0.00000000000,0.00000000000,0.00000000000,0.00000000000,2.52083333333,0.00000000000,0.50000000000,2.30769230769
50%,4.85000000000,0.00000000000,0.00000000000,0.00000000000,0.00000000000,0.00000000000,3.75000000000,0.00000000000,7.41666666667,3.26923076923
75%,6.20000000000,0.00000000000,0.00000000000,2.53333333333,0.00000000000,3.22727272727,6.66666666667,0.00000000000,9.83333333333,4.75000000000
max,13.20000000000,2.80000000000,18.75000000000,9.66666666667,17.33333333333,15.72727272727,19.91666666667,14.60000000000,22.00000000000,9.92307692308


In [7]:
pandas.set_option("display.precision", 11)
arm1_db_df = arm1_df.join(patients_log.set_index('PatientFirstName'), on='PatientFirstName')
print(f"{arm1_db_df[['dpfs'] + [pw[0] for pw in pathways]].corr().iloc[0]}")

dpfs          1.00000000000
Cell Cycle   -0.12647532371
HIPPO         0.07618156944
MYC          -0.01966932520
NOTCH        -0.00766043272
NRF2          0.11540925246
PI3K          0.00471179680
RTK-RAS       0.13577007518
TGF-Beta     -0.08225417760
TP53         -0.04168762758
WNT          -0.00075981504
Name: dpfs, dtype: float64


## Conversion of pathway data
Pathways are parsed from pathway files shipped with [PathwayMapper](http://www.pathwaymapper.org/). At this stage, the obtained data is transformed into a NetworkX graph giving each gene its own vertex: complexes and families are not represented explicitly. The resulting graph is directed.

In [8]:
import networkx as nx
import pathways_nx as pnx
import matplotlib.pyplot as plt
import pylab
import logging as log

plt.rcParams['figure.dpi'] = 90

@widgets.interact(pathway=[filename for filename in os.listdir('./pathways')])
def show_pathway(pathway):
    pw = pnx.pathway_to_nx('pathways/' + pathway)

    edge_labels=dict([((u, v,), d['label']) for u, v, d in pw[1].edges(data=True)])
    labels = nx.get_node_attributes(pw[1], 'label')
    pos=nx.spring_layout(pw[1], 1.5 * len(pw[1]))

    plt.figure(1, figsize=(10, 10)) 
    nx.draw_networkx_edge_labels(pw[1], pos, edge_labels=edge_labels)
    nx.draw(pw[1], pos, node_size=1700, labels=labels, with_labels=True, node_shape="o",  node_color="none", bbox=dict(facecolor="skyblue", edgecolor='black', boxstyle='round,pad=0.4'))

interactive(children=(Dropdown(description='pathway', options=('TGF-Beta.txt', 'HIPPO.txt', 'WNT.txt', 'NRF2.t…

## Computing weighted averages with no complexes
In order to improve the correlation between **dpfs** and pathway mutations, we can employ weights on each gene.</br>
These are derived from various centrality measures and don't take into account the gene hierarchy.

In [9]:
import pathways_nx as pnx
import networkx as nx
import os
import pandas

nx_pathways = []
for filename in os.listdir('./pathways'):
    nx_pathways.append(pnx.pathway_to_nx('pathways/' + filename))

nx_pathways.sort(key=lambda x: x[0])
    
patients_log = pandas.read_csv('TRIBE2_db.csv')
mutations_data = pandas.read_csv('TRIBE2_seq_res.csv')

### In-degree

In [10]:
from analysis_nx import process_patients_with_f

arm0_df_indeg = process_patients_with_f(patients_log[patients_log['arm'] == 0]['PatientFirstName'], nx.in_degree_centrality, nx_pathways, mutations_data)
arm0_df_indeg.describe()

,Cell Cycle,HIPPO,MYC,NOTCH,NRF2,PI3K,RTK-RAS,TGF-Beta,TP53,WNT
count,162.00000000000,162.0,162.0,162.00000000000,162.00000000000,162.00000000000,162.00000000000,162.00000000000,162.00000000000,162.00000000000
mean,9.70370370370,0.0,0.0,1.16352201258,0.91358024691,2.35570987654,1.49255002129,2.62692901235,18.76388888889,0.65506172840
std,6.56313238433,0.0,0.0,2.29956675709,5.59879652300,3.61499308582,1.04267967870,4.95305711540,12.26853237539,1.42037710281
min,0.00000000000,0.0,0.0,0.00000000000,0.00000000000,0.00000000000,0.00000000000,0.00000000000,0.00000000000,0.00000000000
25%,5.75000000000,0.0,0.0,0.00000000000,0.00000000000,0.00000000000,0.82758620690,0.00000000000,10.12500000000,0.00000000000
50%,10.75000000000,0.0,0.0,0.00000000000,0.00000000000,0.00000000000,1.36206896552,0.00000000000,21.25000000000,0.00000000000
75%,14.00000000000,0.0,0.0,0.61320754717,0.00000000000,4.53125000000,2.10344827586,2.26562500000,27.50000000000,0.00000000000
max,30.00000000000,0.0,0.0,11.09433962264,44.00000000000,12.70833333333,4.78160919540,20.75000000000,43.00000000000,7.34000000000


In [11]:
pandas.set_option("display.precision", 11)
arm0_db_d_indegf = arm0_df_indeg.join(patients_log.set_index('PatientFirstName'), on='PatientFirstName')
print(f"{arm0_db_d_indegf[['dpfs'] + [pw[0] for pw in nx_pathways]].corr().iloc[0]}")

dpfs          1.00000000000
Cell Cycle   -0.09226936015
HIPPO                   NaN
MYC                     NaN
NOTCH         0.07403685237
NRF2          0.26055653829
PI3K          0.03503757492
RTK-RAS      -0.11310601928
TGF-Beta     -0.05180275226
TP53         -0.05435628743
WNT          -0.02174170470
Name: dpfs, dtype: float64


In [12]:
from analysis_nx import process_patients_with_f

arm1_df_indeg = process_patients_with_f(patients_log[patients_log['arm'] == 1]['PatientFirstName'], nx.in_degree_centrality, nx_pathways, mutations_data)
arm1_df_indeg.describe()

,Cell Cycle,HIPPO,MYC,NOTCH,NRF2,PI3K,RTK-RAS,TGF-Beta,TP53,WNT
count,134.00000000000,134.0,134.00000000000,134.00000000000,134.00000000000,134.00000000000,134.00000000000,134.00000000000,134.00000000000,134.00000000000
mean,8.92723880597,0.0,0.03616532721,1.11179949310,0.05970149254,2.22388059701,1.68811116830,1.63386194030,17.61473880597,0.66671641791
std,6.72526270743,0.0,0.25563721307,2.13433296243,0.69109474047,3.71690587683,1.17443857065,4.22081387824,13.64440010607,1.39379493649
min,0.00000000000,0.0,0.00000000000,0.00000000000,0.00000000000,0.00000000000,0.00000000000,0.00000000000,0.00000000000,0.00000000000
25%,0.00000000000,0.0,0.00000000000,0.00000000000,0.00000000000,0.00000000000,0.93103448276,0.00000000000,0.00000000000,0.00000000000
50%,10.62500000000,0.0,0.00000000000,0.00000000000,0.00000000000,0.00000000000,1.51724137931,0.00000000000,20.75000000000,0.00000000000
75%,13.75000000000,0.0,0.00000000000,0.81132075472,0.00000000000,4.12500000000,2.27586206897,0.00000000000,27.50000000000,0.00000000000
max,21.00000000000,0.0,2.19230769231,10.15094339623,8.00000000000,17.91666666667,5.83908045977,18.25000000000,44.12500000000,7.54000000000


In [13]:
pandas.set_option("display.precision", 11)
arm1_db_df_indeg = arm1_df_indeg.join(patients_log.set_index('PatientFirstName'), on='PatientFirstName')
print(f"{arm1_db_df_indeg[['dpfs'] + [pw[0] for pw in nx_pathways]].corr().iloc[0]}")

dpfs          1.00000000000
Cell Cycle   -0.12639137193
HIPPO                   NaN
MYC          -0.05265988914
NOTCH        -0.02592021718
NRF2          0.08352453560
PI3K          0.13209202648
RTK-RAS       0.14794631496
TGF-Beta     -0.06616595990
TP53         -0.11281684289
WNT          -0.05256678378
Name: dpfs, dtype: float64


### Out-degree

In [14]:
from analysis_nx import process_patients_with_f

arm0_df_outdeg = process_patients_with_f(patients_log[patients_log['arm'] == 0]['PatientFirstName'], nx.out_degree_centrality, nx_pathways, mutations_data)
arm0_df_outdeg.describe()

,Cell Cycle,HIPPO,MYC,NOTCH,NRF2,PI3K,RTK-RAS,TGF-Beta,TP53,WNT
count,162.00000000000,162.00000000000,162.00000000000,162.00000000000,162.00000000000,162.00000000000,162.00000000000,162.00000000000,162.00000000000,162.0000000000
mean,10.43904320988,0.01037037037,0.11490978158,0.70288842301,0.16666666667,1.47325102881,1.51035901802,0.19097222222,0.82330246914,0.9837037037
std,6.42322427630,0.07741867213,0.46489043298,1.52381859048,1.49948231232,2.16942021878,1.86145914401,1.15780437321,2.40214727723,0.9006504819
min,0.00000000000,0.00000000000,0.00000000000,0.00000000000,0.00000000000,0.00000000000,0.00000000000,0.00000000000,0.00000000000,0.0000000000
25%,5.75000000000,0.00000000000,0.00000000000,0.00000000000,0.00000000000,0.00000000000,0.82758620690,0.00000000000,0.00000000000,0.4850000000
50%,11.50000000000,0.00000000000,0.00000000000,0.00000000000,0.00000000000,0.00000000000,1.25287356322,0.00000000000,0.00000000000,0.7800000000
75%,14.84375000000,0.00000000000,0.00000000000,0.00000000000,0.00000000000,2.75000000000,1.75000000000,0.00000000000,0.00000000000,1.2800000000
max,25.00000000000,0.68000000000,2.69230769231,6.81132075472,14.50000000000,11.08333333333,16.58620689655,9.93750000000,16.50000000000,5.2800000000


In [15]:
pandas.set_option("display.precision", 11)
arm0_db_df_outdeg = arm0_df_outdeg.join(patients_log.set_index('PatientFirstName'), on='PatientFirstName')
print(f"{arm0_db_df_outdeg[['dpfs'] + [pw[0] for pw in nx_pathways]].corr().iloc[0]}")

dpfs          1.00000000000
Cell Cycle   -0.04413363227
HIPPO         0.04245896105
MYC           0.26819358259
NOTCH         0.06066446533
NRF2         -0.04613419523
PI3K          0.09017649935
RTK-RAS      -0.09855698721
TGF-Beta      0.02890916087
TP53          0.00559572918
WNT           0.09316713412
Name: dpfs, dtype: float64


In [16]:
from analysis_nx import process_patients_with_f

arm1_df_outdeg = process_patients_with_f(patients_log[patients_log['arm'] == 1]['PatientFirstName'], nx.out_degree_centrality, nx_pathways, mutations_data)
arm1_df_outdeg.describe()

,Cell Cycle,HIPPO,MYC,NOTCH,NRF2,PI3K,RTK-RAS,TGF-Beta,TP53,WNT
count,134.00000000000,134.00000000000,134.00000000000,134.00000000000,134.00000000000,134.00000000000,134.00000000000,134.00000000000,134.00000000000,134.00000000000
mean,9.61660447761,0.01253731343,0.20034443169,0.69050971557,0.69402985075,1.42133084577,1.83136043918,0.32322761194,0.62313432836,0.97701492537
std,6.81259610129,0.10784761295,0.68159037286,1.50015255307,3.79993591545,2.27909035171,2.38327426401,1.56543007449,2.32614396399,0.94011519839
min,0.00000000000,0.00000000000,0.00000000000,0.00000000000,0.00000000000,0.00000000000,0.00000000000,0.00000000000,0.00000000000,0.00000000000
25%,2.93750000000,0.00000000000,0.00000000000,0.00000000000,0.00000000000,0.00000000000,0.81034482759,0.00000000000,0.00000000000,0.52000000000
50%,11.12500000000,0.00000000000,0.00000000000,0.00000000000,0.00000000000,0.00000000000,1.33908045977,0.00000000000,0.00000000000,0.77000000000
75%,14.75000000000,0.00000000000,0.00000000000,0.00000000000,0.00000000000,2.31250000000,2.00000000000,0.00000000000,0.00000000000,1.19000000000
max,24.37500000000,1.12000000000,4.38461538462,7.94339622642,26.00000000000,10.75000000000,14.68965517241,10.31250000000,20.50000000000,5.32000000000


In [17]:
pandas.set_option("display.precision", 11)
arm1_db_df_outdeg = arm1_df_outdeg.join(patients_log.set_index('PatientFirstName'), on='PatientFirstName')
print(f"{arm1_db_df_outdeg[['dpfs'] + [pw[0] for pw in nx_pathways]].corr().iloc[0]}")

dpfs          1.00000000000
Cell Cycle   -0.13821505997
HIPPO         0.07618156944
MYC          -0.04663580626
NOTCH         0.08720395888
NRF2          0.10812386832
PI3K          0.07505846557
RTK-RAS      -0.02274674199
TGF-Beta     -0.06184457498
TP53          0.06770914810
WNT           0.01182285336
Name: dpfs, dtype: float64


### Betweenness

In [18]:
from analysis_nx import process_patients_with_f

arm0_df_bet = process_patients_with_f(patients_log[patients_log['arm'] == 0]['PatientFirstName'], nx.betweenness_centrality, nx_pathways, mutations_data)
arm0_df_bet.describe()

/home/eli/Documents/pancan/analysis_nx.py:31: RuntimeWarning: invalid value encountered in double_scalars
  perc_mutation = weights.mul(patient_mutations, fill_value=np.float64(0.0)).sum() / weights.sum()


,Cell Cycle,HIPPO,MYC,NOTCH,NRF2,PI3K,RTK-RAS,TGF-Beta,TP53,WNT
count,162.00000000000,162.0,162.0,162.00000000000,155.0,162.00000000000,162.00000000000,162.00000000000,162.00000000000,162.00000000000
mean,18.60141093474,0.0,0.0,1.12500000000,0.0,2.45004572474,2.22161352986,0.25462962963,0.33641975309,0.17548500882
std,12.28116269145,0.0,0.0,3.50700828953,0.0,4.10633802364,1.47314564193,1.54373916427,2.53168139873,0.58189353177
min,0.00000000000,0.0,0.0,0.00000000000,0.0,0.00000000000,0.00000000000,0.00000000000,0.00000000000,0.00000000000
25%,10.17857142857,0.0,0.0,0.00000000000,0.0,0.00000000000,1.37561924982,0.00000000000,0.00000000000,0.00000000000
50%,20.75000000000,0.0,0.0,0.00000000000,0.0,0.00000000000,2.34394904459,0.00000000000,0.00000000000,0.00000000000
75%,27.50000000000,0.0,0.0,0.00000000000,0.0,5.00000000000,3.14968152866,0.00000000000,0.00000000000,0.00000000000
max,43.00000000000,0.0,0.0,18.75000000000,0.0,16.94444444444,6.56121726822,13.25000000000,22.50000000000,3.09523809524


In [19]:
pandas.set_option("display.precision", 11)
arm0_db_df_bet = arm0_df_bet.join(patients_log.set_index('PatientFirstName'), on='PatientFirstName')
print(f"{arm0_db_df_bet[['dpfs'] + [pw[0] for pw in nx_pathways]].corr().iloc[0]}")

dpfs          1.00000000000
Cell Cycle   -0.05919601305
HIPPO                   NaN
MYC                     NaN
NOTCH         0.06796449064
NRF2                    NaN
PI3K         -0.00252672381
RTK-RAS      -0.10656893393
TGF-Beta      0.02890916087
TP53         -0.04865032728
WNT           0.01694620962
Name: dpfs, dtype: float64


In [20]:
from analysis_nx import process_patients_with_f

arm1_df_bet = process_patients_with_f(patients_log[patients_log['arm'] == 1]['PatientFirstName'], nx.betweenness_centrality, nx_pathways, mutations_data)
arm1_df_bet.describe()

/home/eli/Documents/pancan/analysis_nx.py:31: RuntimeWarning: invalid value encountered in double_scalars
  perc_mutation = weights.mul(patient_mutations, fill_value=np.float64(0.0)).sum() / weights.sum()


,Cell Cycle,HIPPO,MYC,NOTCH,NRF2,PI3K,RTK-RAS,TGF-Beta,TP53,WNT
count,134.00000000000,134.0,134.00000000000,134.00000000000,128.0,134.00000000000,134.00000000000,134.00000000000,134.00000000000,134.00000000000
mean,17.45522388060,0.0,0.21699196326,1.75373134328,0.0,2.28784894970,2.33660078940,0.43097014925,0.63805970149,0.08742004264
std,13.60139625276,0.0,1.53382327841,4.72445496266,0.0,4.16056070500,1.54687210060,2.08724009931,3.58641536277,0.41345394441
min,0.00000000000,0.0,0.00000000000,0.00000000000,0.0,0.00000000000,0.00000000000,0.00000000000,0.00000000000,0.00000000000
25%,0.00000000000,0.0,0.00000000000,0.00000000000,0.0,0.00000000000,1.40207006369,0.00000000000,0.00000000000,0.00000000000
50%,20.75000000000,0.0,0.00000000000,0.00000000000,0.0,0.00000000000,2.36883698986,0.00000000000,0.00000000000,0.00000000000
75%,27.50000000000,0.0,0.00000000000,0.00000000000,0.0,4.09722222222,3.25123849965,0.00000000000,0.00000000000,0.00000000000
max,42.00000000000,0.0,13.15384615385,22.75000000000,0.0,23.88888888889,6.46956829441,13.75000000000,27.50000000000,2.52380952381


In [21]:
pandas.set_option("display.precision", 11)
arm1_db_df_bet = arm1_df_bet.join(patients_log.set_index('PatientFirstName'), on='PatientFirstName')
print(f"{arm1_db_df_bet[['dpfs'] + [pw[0] for pw in nx_pathways]].corr().iloc[0]}")

dpfs          1.00000000000
Cell Cycle   -0.12418602217
HIPPO                   NaN
MYC          -0.05265988914
NOTCH         0.05031069590
NRF2                    NaN
PI3K          0.17710123936
RTK-RAS       0.12833767829
TGF-Beta     -0.06184457498
TP53          0.16705834475
WNT           0.10809996755
Name: dpfs, dtype: float64


### Closeness

In [22]:
from analysis_nx import process_patients_with_f

arm0_df_clos = process_patients_with_f(patients_log[patients_log['arm'] == 0]['PatientFirstName'], nx.closeness_centrality, nx_pathways, mutations_data)
arm0_df_clos.describe()

,Cell Cycle,HIPPO,MYC,NOTCH,NRF2,PI3K,RTK-RAS,TGF-Beta,TP53,WNT
count,162.00000000000,162.0,162.0,162.00000000000,162.00000000000,162.00000000000,162.00000000000,162.00000000000,162.00000000000,162.00000000000
mean,8.70921985816,0.0,0.0,1.08187134503,0.91358024691,1.84509887854,2.45586968214,2.69470899471,18.01333333333,0.73695257663
std,5.94299134480,0.0,0.0,2.13819365133,5.59879652300,2.82678671450,1.55701006177,5.09432734482,11.77779108037,1.65284382062
min,0.00000000000,0.0,0.0,0.00000000000,0.00000000000,0.00000000000,0.00000000000,0.00000000000,0.00000000000,0.00000000000
25%,5.13829787234,0.0,0.0,0.00000000000,0.00000000000,0.00000000000,1.50727423468,0.00000000000,9.72000000000,0.00000000000
50%,9.60638297872,0.0,0.0,0.00000000000,0.00000000000,0.00000000000,2.52261371738,0.00000000000,20.40000000000,0.00000000000
75%,12.51063829787,0.0,0.0,0.57017543860,0.00000000000,3.62285452697,3.44371986304,2.28571428571,26.40000000000,0.00000000000
max,27.86170212766,0.0,0.0,10.31578947368,44.00000000000,13.49997324661,7.35848279182,21.34285714286,41.28000000000,8.09784126829


In [23]:
pandas.set_option("display.precision", 11)
arm0_db_df_clos = arm0_df_clos.join(patients_log.set_index('PatientFirstName'), on='PatientFirstName')
print(f"{arm0_db_df_clos[['dpfs'] + [pw[0] for pw in nx_pathways]].corr().iloc[0]}")

dpfs          1.00000000000
Cell Cycle   -0.09395603602
HIPPO                   NaN
MYC                     NaN
NOTCH         0.07403685237
NRF2          0.26055653829
PI3K          0.09336284236
RTK-RAS      -0.11431322190
TGF-Beta     -0.05205554694
TP53         -0.05435628743
WNT          -0.01677642636
Name: dpfs, dtype: float64


In [24]:
from analysis_nx import process_patients_with_f

arm1_df_clos = process_patients_with_f(patients_log[patients_log['arm'] == 1]['PatientFirstName'], nx.closeness_centrality, nx_pathways, mutations_data)
arm1_df_clos.describe()

,Cell Cycle,HIPPO,MYC,NOTCH,NRF2,PI3K,RTK-RAS,TGF-Beta,TP53,WNT
count,134.00000000000,134.0,134.00000000000,134.00000000000,134.00000000000,134.00000000000,134.00000000000,134.00000000000,134.00000000000,134.00000000000
mean,8.00301683074,0.0,0.02801717102,1.03377847604,0.05970149254,1.76261093893,2.63326260969,1.66823027719,16.91014925373,0.63744556724
std,6.02711334300,0.0,0.19804138575,1.98455521068,0.69109474047,2.85228658469,1.66904695495,4.33893048562,13.09862410183,1.38394166607
min,0.00000000000,0.0,0.00000000000,0.00000000000,0.00000000000,0.00000000000,0.00000000000,0.00000000000,0.00000000000,0.00000000000
25%,0.00000000000,0.0,0.00000000000,0.00000000000,0.00000000000,0.00000000000,1.59733983042,0.00000000000,0.00000000000,0.00000000000
50%,9.49468085106,0.0,0.00000000000,0.00000000000,0.00000000000,0.00000000000,2.72852839314,0.00000000000,19.92000000000,0.00000000000
75%,12.28723404255,0.0,0.00000000000,0.75438596491,0.00000000000,3.35717852833,3.60695492537,0.00000000000,26.40000000000,0.00000000000
max,18.76595744681,0.0,1.69837422406,9.43859649123,8.00000000000,12.78351753536,7.73485102251,18.77142857143,42.36000000000,6.87745950847


In [25]:
pandas.set_option("display.precision", 11)
arm1_db_df_clos = arm1_df_clos.join(patients_log.set_index('PatientFirstName'), on='PatientFirstName')
print(f"{arm1_db_df_clos[['dpfs'] + [pw[0] for pw in nx_pathways]].corr().iloc[0]}")

dpfs          1.00000000000
Cell Cycle   -0.12614421901
HIPPO                   NaN
MYC          -0.05265988914
NOTCH        -0.02592021718
NRF2          0.08352453560
PI3K          0.07809505960
RTK-RAS       0.14198272352
TGF-Beta     -0.06535374045
TP53         -0.11281684289
WNT          -0.00141656123
Name: dpfs, dtype: float64


### Eigenvector

In [27]:
from analysis_nx import process_patients_with_f

arm0_df_eigen = process_patients_with_f(patients_log[patients_log['arm'] == 0]['PatientFirstName'], nx.eigenvector_centrality_numpy, nx_pathways, mutations_data)
arm0_df_eigen.describe()

,Cell Cycle,HIPPO,MYC,NOTCH,NRF2,PI3K,RTK-RAS,TGF-Beta,TP53,WNT
count,1.62000000000e+02,1.62000000000e+02,1.62000000000e+02,162.00000000000,162.00000000000,1.62000000000e+02,162.00000000000,162.00000000000,162.00000000000,1.62000000000e+02
mean,9.70370370370e+00,-1.62441141528e-14,9.78181885374e-14,0.00000196813,0.72839506227,2.06567833958e+00,0.00006587604,3.41769249239,36.88077764262,1.87011749567e+00
std,6.56313238433e+00,4.77876957631e-13,8.09244706963e-13,0.00001162848,5.34292901613,7.15872256933e+00,0.00017736779,6.61900653205,24.73268698762,8.73103422215e+00
min,-1.27675647832e-15,-5.39882401073e-12,-5.64195876162e-13,-0.00008036315,0.00000000000,-1.99309453066e-10,-0.00000000181,-0.00000583567,-0.00000010722,-3.13020064061e-09
25%,5.75000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000,0.00000000000,0.00000000000e+00,0.00000000003,0.00000000000,20.24805636379,-6.71120277885e-13
50%,1.07500000000e+01,0.00000000000e+00,0.00000000000e+00,0.00000000000,0.00000000000,0.00000000000e+00,0.00000035921,0.00000000000,40.99745867952,0.00000000000e+00
75%,1.40000000000e+01,0.00000000000e+00,0.00000000000e+00,0.00000000006,0.00000000000,8.37714554077e-13,0.00000190928,0.00001368201,54.99699898134,9.61068921222e-13
max,3.00000000000e+01,2.76811018523e-12,9.32307611490e-12,0.00007097734,43.99999981037,3.64562390842e+01,0.00104536064,27.66660782115,85.99714836184,5.79572704292e+01


In [28]:
pandas.set_option("display.precision", 11)
arm0_db_df_eigen = arm0_df_eigen.join(patients_log.set_index('PatientFirstName'), on='PatientFirstName')
print(f"{arm0_db_df_eigen[['dpfs'] + [pw[0] for pw in nx_pathways]].corr().iloc[0]}")

dpfs          1.00000000000
Cell Cycle   -0.09226936015
HIPPO         0.20563209657
MYC           0.16280776071
NOTCH        -0.01225861307
NRF2          0.19523889819
PI3K          0.11202159772
RTK-RAS      -0.09990320319
TGF-Beta     -0.05393329275
TP53         -0.04579503174
WNT          -0.05336089500
Name: dpfs, dtype: float64


In [29]:
from analysis_nx import process_patients_with_f

arm1_df_eigen = process_patients_with_f(patients_log[patients_log['arm'] == 1]['PatientFirstName'], nx.eigenvector_centrality_numpy, nx_pathways, mutations_data)
arm1_df_eigen.describe()

,Cell Cycle,HIPPO,MYC,NOTCH,NRF2,PI3K,RTK-RAS,TGF-Beta,TP53,WNT
count,1.34000000000e+02,1.34000000000e+02,1.34000000000e+02,1.34000000000e+02,134.00000000000,134.00000000000,134.00000000000,1.34000000000e+02,134.00000000000,1.34000000000e+02
mean,8.92723880597e+00,-5.92106007870e-14,-1.33550190013e-11,1.06646365465e-06,-0.27841978970,1.72184880227,0.00009407370,2.03482416117e+00,34.90871272440,1.67849325181e+00
std,6.72526270743e+00,6.17189315910e-13,9.10988656338e-11,1.21712048824e-05,5.44881032728,6.06870720125,0.00023141887,5.63706316563e+00,27.20139982628,8.10350876383e+00
min,-3.19189119580e-15,-7.10212023045e-12,-6.98257846149e-10,-9.52090602247e-05,-60.52678450791,-0.00000000026,-0.00000001789,-3.59594022354e-14,-0.00000000001,-4.20408175659e-09
25%,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000,0.00000000000,0.00000000170,0.00000000000e+00,0.00000000000,-4.81500049258e-13
50%,1.06250000000e+01,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000,0.00000000000,0.00000040429,0.00000000000e+00,41.49690972657,4.69237524831e-14
75%,1.37500000000e+01,0.00000000000e+00,0.00000000000e+00,4.90431406774e-12,0.00000000000,0.00000000004,0.00000262629,0.00000000000e+00,54.99711666399,1.20978796980e-12
max,2.10000000000e+01,0.00000000000e+00,5.17372712191e-12,3.21627149974e-05,15.21853253282,37.96897803393,0.00131125981,2.43333248382e+01,83.99812610221,5.79677497387e+01


In [30]:
pandas.set_option("display.precision", 11)
arm1_db_df_eigen = arm1_df_eigen.join(patients_log.set_index('PatientFirstName'), on='PatientFirstName')
print(f"{arm1_db_df_eigen[['dpfs'] + [pw[0] for pw in nx_pathways]].corr().iloc[0]}")

dpfs          1.00000000000
Cell Cycle   -0.12639137193
HIPPO         0.04952601001
MYC           0.03030567971
NOTCH         0.05996250180
NRF2          0.10301857717
PI3K         -0.00634697178
RTK-RAS       0.09052514646
TGF-Beta     -0.05842371545
TP53         -0.12418280026
WNT          -0.00276112921
Name: dpfs, dtype: float64


## Hierarchy-aware evaluation
This time the hierarchy of a gene inside a complex or family is taken into account when computing the average mutations. This is represented with a weight, which is computed as the reciprocal of the product of the gene containers' cardinalities. For example, if a gene is contained in a family of 4, which is contained in a family of 6, it would have a weight of 1/4\*1/6 = 1/24.

In [31]:
import pathways_nx as pnx
import networkx as nx
import os
import pandas

nx_pathways = []
for filename in os.listdir('./pathways'):
    nx_pathways.append(pnx.pathway_to_nx('pathways/' + filename))

nx_pathways.sort(key=lambda x: x[0])
    
patients_log = pandas.read_csv('TRIBE2_db.csv')
mutations_data = pandas.read_csv('TRIBE2_seq_res.csv')

### In-degree

In [32]:
from analysis_nx import process_patients_with_f

h_arm0_df_indeg = process_patients_with_f(patients_log[patients_log['arm'] == 0]['PatientFirstName'], nx.in_degree_centrality, nx_pathways, mutations_data, True)
h_arm0_df_indeg.describe()

,Cell Cycle,HIPPO,MYC,NOTCH,NRF2,PI3K,RTK-RAS,TGF-Beta,TP53,WNT
count,162.00000000000,162.0,162.0,162.00000000000,162.00000000000,162.00000000000,162.00000000000,162.00000000000,162.00000000000,162.00000000000
mean,9.70370370370,0.0,0.0,0.85265700483,0.91358024691,2.95500685871,0.86468595419,2.36334019204,21.30820105820,0.70694253925
std,6.56313238433,0.0,0.0,1.52048866488,5.59879652300,4.79080843015,0.62285094125,4.40775376490,14.03997938684,1.83873948384
min,0.00000000000,0.0,0.0,0.00000000000,0.00000000000,0.00000000000,0.00000000000,0.00000000000,0.00000000000,0.00000000000
25%,5.75000000000,0.0,0.0,0.00000000000,0.00000000000,0.00000000000,0.54545454545,0.00000000000,11.57142857143,0.00000000000
50%,10.75000000000,0.0,0.0,0.00000000000,0.00000000000,0.00000000000,0.81860269360,0.00000000000,24.28571428571,0.00000000000
75%,14.00000000000,0.0,0.0,1.30434782609,0.00000000000,5.92500000000,1.17445286195,2.18750000000,31.42857142857,0.00000000000
max,30.00000000000,0.0,0.0,7.30434782609,44.00000000000,20.33333333333,3.11784511785,18.44444444444,49.14285714286,10.74074074074


In [33]:
pandas.set_option("display.precision", 11)
h_arm0_db_d_indegf = h_arm0_df_indeg.join(patients_log.set_index('PatientFirstName'), on='PatientFirstName')
print(f"{h_arm0_db_d_indegf[['dpfs'] + [pw[0] for pw in nx_pathways]].corr().iloc[0]}")

dpfs          1.00000000000
Cell Cycle   -0.09226936015
HIPPO                   NaN
MYC                     NaN
NOTCH         0.05404772121
NRF2          0.26055653829
PI3K         -0.00620778660
RTK-RAS      -0.11536385382
TGF-Beta     -0.05061856973
TP53         -0.05144321928
WNT          -0.03280731563
Name: dpfs, dtype: float64


In [34]:
from analysis_nx import process_patients_with_f

h_arm1_df_indeg = process_patients_with_f(patients_log[patients_log['arm'] == 1]['PatientFirstName'], nx.in_degree_centrality, nx_pathways, mutations_data, True)
h_arm1_df_indeg.describe()

,Cell Cycle,HIPPO,MYC,NOTCH,NRF2,PI3K,RTK-RAS,TGF-Beta,TP53,WNT
count,134.00000000000,134.0,134.00000000000,134.00000000000,134.00000000000,134.00000000000,134.00000000000,134.00000000000,134.00000000000,134.00000000000
mean,8.92723880597,0.0,0.05424799082,0.76216742375,0.05970149254,2.69469320066,0.95453289110,1.50020729685,20.13113006397,0.65284687673
std,6.72526270743,0.0,0.38345581960,1.31734227102,0.69109474047,4.90924707032,0.70336646216,3.77018454091,15.59360012122,1.69371953919
min,0.00000000000,0.0,0.00000000000,0.00000000000,0.00000000000,0.00000000000,0.00000000000,0.00000000000,0.00000000000,0.00000000000
25%,0.00000000000,0.0,0.00000000000,0.00000000000,0.00000000000,0.00000000000,0.52840909091,0.00000000000,0.00000000000,0.00000000000
50%,10.62500000000,0.0,0.00000000000,0.00000000000,0.00000000000,0.00000000000,0.89772727273,0.00000000000,23.71428571429,0.00000000000
75%,13.75000000000,0.0,0.00000000000,1.21195652174,0.00000000000,3.96666666667,1.26262626263,0.00000000000,31.42857142857,0.00000000000
max,21.00000000000,0.0,3.28846153846,5.84782608696,8.00000000000,28.66666666667,4.16161616162,16.22222222222,50.42857142857,10.74074074074


In [35]:
pandas.set_option("display.precision", 11)
h_arm1_db_df_indeg = h_arm1_df_indeg.join(patients_log.set_index('PatientFirstName'), on='PatientFirstName')
print(f"{h_arm1_db_df_indeg[['dpfs'] + [pw[0] for pw in nx_pathways]].corr().iloc[0]}")

dpfs          1.00000000000
Cell Cycle   -0.12639137193
HIPPO                   NaN
MYC          -0.05265988914
NOTCH        -0.05638212697
NRF2          0.08352453560
PI3K          0.18505394286
RTK-RAS       0.12814849061
TGF-Beta     -0.06964817020
TP53         -0.11281684289
WNT          -0.01808717964
Name: dpfs, dtype: float64


### Out-degree

In [36]:
from analysis_nx import process_patients_with_f

h_arm0_df_outdeg = process_patients_with_f(patients_log[patients_log['arm'] == 0]['PatientFirstName'], nx.out_degree_centrality, nx_pathways, mutations_data, True)
h_arm0_df_outdeg.describe()

,Cell Cycle,HIPPO,MYC,NOTCH,NRF2,PI3K,RTK-RAS,TGF-Beta,TP53,WNT
count,162.00000000000,162.00000000000,162.00000000000,162.00000000000,162.00000000000,162.00000000000,162.00000000000,162.00000000000,162.00000000000,162.00000000000
mean,10.43904320988,0.00997150997,0.12989801396,0.29103973765,0.16666666667,1.79744613895,0.93811238487,0.15277777778,0.80467372134,2.25587889477
std,6.42322427630,0.07444103089,0.52552831554,0.63095613512,1.49948231232,2.75267087997,2.64679131601,0.92624349856,2.48929503244,2.06658657491
min,0.00000000000,0.00000000000,0.00000000000,0.00000000000,0.00000000000,0.00000000000,0.00000000000,0.00000000000,0.00000000000,0.00000000000
25%,5.75000000000,0.00000000000,0.00000000000,0.00000000000,0.00000000000,0.00000000000,0.29241071429,0.00000000000,0.00000000000,1.14285714286
50%,11.50000000000,0.00000000000,0.00000000000,0.00000000000,0.00000000000,0.00000000000,0.46701388889,0.00000000000,0.00000000000,1.85714285714
75%,14.84375000000,0.00000000000,0.00000000000,0.00000000000,0.00000000000,3.17401960784,0.65290178571,0.00000000000,0.00000000000,2.95238095238
max,25.00000000000,0.65384615385,3.04347826087,2.82031250000,14.50000000000,12.16666666667,24.01884920635,7.95000000000,18.85714285714,12.57142857143


In [37]:
pandas.set_option("display.precision", 11)
h_arm0_db_df_outdeg = h_arm0_df_outdeg.join(patients_log.set_index('PatientFirstName'), on='PatientFirstName')
print(f"{h_arm0_db_df_outdeg[['dpfs'] + [pw[0] for pw in nx_pathways]].corr().iloc[0]}")

dpfs          1.00000000000
Cell Cycle   -0.04413363227
HIPPO         0.04245896105
MYC           0.26819358259
NOTCH         0.06066446533
NRF2         -0.04613419523
PI3K          0.04951808483
RTK-RAS      -0.07037608918
TGF-Beta      0.02890916087
TP53          0.02219084567
WNT           0.09687513852
Name: dpfs, dtype: float64


In [38]:
from analysis_nx import process_patients_with_f

h_arm1_df_outdeg = process_patients_with_f(patients_log[patients_log['arm'] == 1]['PatientFirstName'], nx.out_degree_centrality, nx_pathways, mutations_data, True)
h_arm1_df_outdeg.describe()

,Cell Cycle,HIPPO,MYC,NOTCH,NRF2,PI3K,RTK-RAS,TGF-Beta,TP53,WNT
count,134.00000000000,134.00000000000,134.00000000000,134.00000000000,134.00000000000,134.00000000000,134.00000000000,134.00000000000,134.00000000000,134.00000000000
mean,9.61660447761,0.01205510907,0.26735885788,0.28591417910,0.69402985075,1.66937371964,1.31226308932,0.25858208955,0.71215351812,2.28251599147
std,6.81259610129,0.10369962784,0.99964471228,0.62115691651,3.79993591545,2.74853184274,3.44522650277,1.25234405959,2.65845024456,2.21676228121
min,0.00000000000,0.00000000000,0.00000000000,0.00000000000,0.00000000000,0.00000000000,0.00000000000,0.00000000000,0.00000000000,0.00000000000
25%,2.93750000000,0.00000000000,0.00000000000,0.00000000000,0.00000000000,0.00000000000,0.31919642857,0.00000000000,0.00000000000,1.20238095238
50%,11.12500000000,0.00000000000,0.00000000000,0.00000000000,0.00000000000,0.00000000000,0.50843253968,0.00000000000,0.00000000000,1.83333333333
75%,14.75000000000,0.00000000000,0.00000000000,0.00000000000,0.00000000000,2.30882352941,0.69456845238,0.00000000000,0.00000000000,2.75000000000
max,24.37500000000,1.07692307692,7.43478260870,3.28906250000,26.00000000000,15.17647058824,22.82142857143,8.25000000000,23.42857142857,12.66666666667


In [39]:
pandas.set_option("display.precision", 11)
h_arm1_db_df_outdeg = h_arm1_df_outdeg.join(patients_log.set_index('PatientFirstName'), on='PatientFirstName')
print(f"{h_arm1_db_df_outdeg[['dpfs'] + [pw[0] for pw in nx_pathways]].corr().iloc[0]}")

dpfs          1.00000000000
Cell Cycle   -0.13821505997
HIPPO         0.07618156944
MYC          -0.05116848131
NOTCH         0.08720395888
NRF2          0.10812386832
PI3K          0.10650559991
RTK-RAS      -0.06684990746
TGF-Beta     -0.06184457498
TP53          0.06770914810
WNT           0.00185709385
Name: dpfs, dtype: float64


### Betweenness

In [40]:
from analysis_nx import process_patients_with_f

h_arm0_df_bet = process_patients_with_f(patients_log[patients_log['arm'] == 0]['PatientFirstName'], nx.betweenness_centrality, nx_pathways, mutations_data, True)
h_arm0_df_bet.describe()

/home/eli/Documents/pancan/analysis_nx.py:31: RuntimeWarning: invalid value encountered in double_scalars
  perc_mutation = weights.mul(patient_mutations, fill_value=np.float64(0.0)).sum() / weights.sum()


,Cell Cycle,HIPPO,MYC,NOTCH,NRF2,PI3K,RTK-RAS,TGF-Beta,TP53,WNT
count,162.00000000000,162.0,162.0,162.00000000000,155.0,162.00000000000,162.00000000000,162.00000000000,162.00000000000,162.00000000000
mean,18.60141093474,0.0,0.0,1.12500000000,0.0,2.97245604190,0.97767708549,0.25462962963,0.33641975309,0.13775417574
std,12.28116269145,0.0,0.0,3.50700828953,0.0,5.39867503644,0.65394810607,1.54373916427,2.53168139873,0.45135975553
min,0.00000000000,0.0,0.0,0.00000000000,0.0,0.00000000000,0.00000000000,0.00000000000,0.00000000000,0.00000000000
25%,10.17857142857,0.0,0.0,0.00000000000,0.0,0.00000000000,0.59783206641,0.00000000000,0.00000000000,0.00000000000
50%,20.75000000000,0.0,0.0,0.00000000000,0.0,0.00000000000,1.01937331911,0.00000000000,0.00000000000,0.00000000000
75%,27.50000000000,0.0,0.0,0.00000000000,0.0,4.09821428571,1.36170428530,0.00000000000,0.00000000000,0.00000000000
max,43.00000000000,0.0,0.0,18.75000000000,0.0,23.76623376623,3.07808228312,13.25000000000,22.50000000000,2.38970588235


In [41]:
pandas.set_option("display.precision", 11)
h_arm0_db_df_bet = h_arm0_df_bet.join(patients_log.set_index('PatientFirstName'), on='PatientFirstName')
print(f"{h_arm0_db_df_bet[['dpfs'] + [pw[0] for pw in nx_pathways]].corr().iloc[0]}")

dpfs          1.00000000000
Cell Cycle   -0.05919601305
HIPPO                   NaN
MYC                     NaN
NOTCH         0.06796449064
NRF2                    NaN
PI3K         -0.02763978883
RTK-RAS      -0.11106173460
TGF-Beta      0.02890916087
TP53         -0.04865032728
WNT           0.03514633955
Name: dpfs, dtype: float64


In [42]:
from analysis_nx import process_patients_with_f

h_arm1_df_bet = process_patients_with_f(patients_log[patients_log['arm'] == 1]['PatientFirstName'], nx.betweenness_centrality, nx_pathways, mutations_data, True)
h_arm1_df_bet.describe()

/home/eli/Documents/pancan/analysis_nx.py:31: RuntimeWarning: invalid value encountered in double_scalars
  perc_mutation = weights.mul(patient_mutations, fill_value=np.float64(0.0)).sum() / weights.sum()


,Cell Cycle,HIPPO,MYC,NOTCH,NRF2,PI3K,RTK-RAS,TGF-Beta,TP53,WNT
count,134.00000000000,134.0,134.00000000000,134.00000000000,128.0,134.00000000000,134.00000000000,134.00000000000,134.00000000000,134.00000000000
mean,17.45522388060,0.0,0.26612221909,1.75373134328,0.0,2.68160819280,1.03715925606,0.43097014925,0.63805970149,0.06749341528
std,13.60139625276,0.0,1.88110402069,4.72445496266,0.0,5.42087517077,0.69955103719,2.08724009931,3.58641536277,0.31921076590
min,0.00000000000,0.0,0.00000000000,0.00000000000,0.0,0.00000000000,0.00000000000,0.00000000000,0.00000000000,0.00000000000
25%,0.00000000000,0.0,0.00000000000,0.00000000000,0.0,0.00000000000,0.65690638128,0.00000000000,0.00000000000,0.00000000000
50%,20.75000000000,0.0,0.00000000000,0.00000000000,0.0,0.00000000000,1.07657364806,0.00000000000,0.00000000000,0.00000000000
75%,27.50000000000,0.0,0.00000000000,0.00000000000,0.0,3.33116883117,1.39752950590,0.00000000000,0.00000000000,0.00000000000
max,42.00000000000,0.0,16.13207547170,22.75000000000,0.0,33.50649350649,3.33786757351,13.75000000000,27.50000000000,1.94852941176


In [43]:
pandas.set_option("display.precision", 11)
h_arm1_db_df_bet = h_arm1_df_bet.join(patients_log.set_index('PatientFirstName'), on='PatientFirstName')
print(f"{h_arm1_db_df_bet[['dpfs'] + [pw[0] for pw in nx_pathways]].corr().iloc[0]}")

dpfs          1.00000000000
Cell Cycle   -0.12418602217
HIPPO                   NaN
MYC          -0.05265988914
NOTCH         0.05031069590
NRF2                    NaN
PI3K          0.20645834212
RTK-RAS       0.12765317155
TGF-Beta     -0.06184457498
TP53          0.16705834475
WNT           0.10809996755
Name: dpfs, dtype: float64


### Closeness

In [44]:
from analysis_nx import process_patients_with_f

h_arm0_df_clos = process_patients_with_f(patients_log[patients_log['arm'] == 0]['PatientFirstName'], nx.closeness_centrality, nx_pathways, mutations_data, True)
h_arm0_df_clos.describe()

,Cell Cycle,HIPPO,MYC,NOTCH,NRF2,PI3K,RTK-RAS,TGF-Beta,TP53,WNT
count,162.00000000000,162.0,162.0,162.00000000000,162.00000000000,162.00000000000,162.00000000000,162.00000000000,162.00000000000,162.00000000000
mean,8.70921985816,0.0,0.0,0.72633744856,0.91358024691,2.21513200322,1.62366815306,2.44444444444,20.33964646465,0.79127111386
std,5.94299134480,0.0,0.0,1.29523108490,5.59879652300,3.27768985009,1.08088562088,4.57476953515,13.40179850562,2.03898084662
min,0.00000000000,0.0,0.0,0.00000000000,0.00000000000,0.00000000000,0.00000000000,0.00000000000,0.00000000000,0.00000000000
25%,5.13829787234,0.0,0.0,0.00000000000,0.00000000000,0.00000000000,0.95964622557,0.00000000000,11.04545454545,0.00000000000
50%,9.60638297872,0.0,0.0,0.00000000000,0.00000000000,0.00000000000,1.69495956723,0.00000000000,23.18181818182,0.00000000000
75%,12.51063829787,0.0,0.0,1.11111111111,0.00000000000,4.56848697754,2.22378629254,2.21153846154,30.00000000000,0.00000000000
max,27.86170212766,0.0,0.0,6.22222222222,44.00000000000,12.18263194011,5.21808371383,19.15384615385,46.90909090909,11.17142356855


In [45]:
pandas.set_option("display.precision", 11)
h_arm0_db_df_clos = h_arm0_df_clos.join(patients_log.set_index('PatientFirstName'), on='PatientFirstName')
print(f"{h_arm0_db_df_clos[['dpfs'] + [pw[0] for pw in nx_pathways]].corr().iloc[0]}")

dpfs          1.00000000000
Cell Cycle   -0.09395603602
HIPPO                   NaN
MYC                     NaN
NOTCH         0.05404772121
NRF2          0.26055653829
PI3K          0.04122717585
RTK-RAS      -0.11385511784
TGF-Beta     -0.05102158129
TP53         -0.05144321928
WNT          -0.02494105771
Name: dpfs, dtype: float64


In [46]:
from analysis_nx import process_patients_with_f

h_arm1_df_clos = process_patients_with_f(patients_log[patients_log['arm'] == 1]['PatientFirstName'], nx.closeness_centrality, nx_pathways, mutations_data, True)
h_arm1_df_clos.describe()

,Cell Cycle,HIPPO,MYC,NOTCH,NRF2,PI3K,RTK-RAS,TGF-Beta,TP53,WNT
count,134.00000000000,134.0,134.00000000000,134.00000000000,134.00000000000,134.00000000000,134.00000000000,134.00000000000,134.00000000000,134.00000000000
mean,8.00301683074,0.0,0.04458794905,0.64925373134,0.05970149254,2.03864652403,1.73607745205,1.54133180253,19.21607869742,0.64845042429
std,6.02711334300,0.0,0.31517312050,1.12218045309,0.69109474047,3.36999319886,1.16864899730,3.90720096733,14.88480011571,1.76334760180
min,0.00000000000,0.0,0.00000000000,0.00000000000,0.00000000000,0.00000000000,0.00000000000,0.00000000000,0.00000000000,0.00000000000
25%,0.00000000000,0.0,0.00000000000,0.00000000000,0.00000000000,0.00000000000,1.10880097120,0.00000000000,0.00000000000,0.00000000000
50%,9.49468085106,0.0,0.00000000000,0.00000000000,0.00000000000,0.00000000000,1.82420659528,0.00000000000,22.63636363636,0.00000000000
75%,12.28723404255,0.0,0.00000000000,1.03240740741,0.00000000000,4.15407777630,2.35348094974,0.00000000000,30.00000000000,0.00000000000
max,18.76595744681,0.0,2.70287900646,4.98148148148,8.00000000000,17.17551388277,5.89668950965,16.84615384615,48.13636363636,11.17142356855


In [47]:
pandas.set_option("display.precision", 11)
h_arm1_db_df_clos = h_arm1_df_clos.join(patients_log.set_index('PatientFirstName'), on='PatientFirstName')
print(f"{h_arm1_db_df_clos[['dpfs'] + [pw[0] for pw in nx_pathways]].corr().iloc[0]}")

dpfs          1.00000000000
Cell Cycle   -0.12614421901
HIPPO                   NaN
MYC          -0.05265988914
NOTCH        -0.05638212697
NRF2          0.08352453560
PI3K          0.15017770745
RTK-RAS       0.13051729697
TGF-Beta     -0.06851993237
TP53         -0.11281684289
WNT           0.01312932282
Name: dpfs, dtype: float64


### Eigenvector

In [48]:
from analysis_nx import process_patients_with_f

h_arm0_df_eigen = process_patients_with_f(patients_log[patients_log['arm'] == 0]['PatientFirstName'], nx.eigenvector_centrality_numpy, nx_pathways, mutations_data, True)
h_arm0_df_eigen.describe()

,Cell Cycle,HIPPO,MYC,NOTCH,NRF2,PI3K,RTK-RAS,TGF-Beta,TP53,WNT
count,1.62000000000e+02,1.62000000000e+02,1.62000000000e+02,162.00000000000,162.00000000000,162.00000000000,162.00000000000,162.00000000000,1.62000000000e+02,1.62000000000e+02
mean,9.70370370370e+00,-6.11438797831e-15,5.16959368824e-14,0.00000087270,0.91358024585,2.06343775056,0.00001944408,3.41768991498,3.68806267505e+01,1.86994774244e+00
std,6.56313238433e+00,6.42735715614e-14,5.12647246798e-13,0.00000773291,5.59879651380,7.15079955274,0.00005443987,6.61900289035,2.47324572657e+01,8.73277365425e+00
min,-4.21884749358e-15,-7.95798808545e-13,-1.73915889668e-12,-0.00003091974,0.00000000000,-0.00000000028,-0.00000000090,-0.00001755317,-5.40682177481e-13,-1.25319407615e-09
25%,5.75000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000,0.00000000000,0.00000000000,0.00000000076,0.00000000000,2.02487205133e+01,-4.48031454073e-13
50%,1.07500000000e+01,0.00000000000e+00,0.00000000000e+00,0.00000000000,0.00000000000,0.00000000000,0.00000008612,0.00000000000,4.09988372347e+01,0.00000000000e+00
75%,1.40000000000e+01,0.00000000000e+00,0.00000000000e+00,0.00000000003,0.00000000000,0.00000000016,0.00000040901,0.00000986736,5.49972765883e+01,9.95266496831e-13
max,3.00000000000e+01,0.00000000000e+00,5.52080489049e-12,0.00007864700,43.99999999568,36.37987125627,0.00038184561,27.66666293959,8.59939438391e+01,5.79821429191e+01


In [49]:
pandas.set_option("display.precision", 11)
h_arm0_db_df_eigen = h_arm0_df_eigen.join(patients_log.set_index('PatientFirstName'), on='PatientFirstName')
print(f"{h_arm0_db_df_eigen[['dpfs'] + [pw[0] for pw in nx_pathways]].corr().iloc[0]}")

dpfs          1.00000000000
Cell Cycle   -0.09226936015
HIPPO         0.04367882613
MYC           0.25121704917
NOTCH         0.17768121281
NRF2          0.26055653771
PI3K          0.11209631948
RTK-RAS       0.03839719012
TGF-Beta     -0.05393343030
TP53         -0.04579173256
WNT          -0.05336344013
Name: dpfs, dtype: float64


In [50]:
from analysis_nx import process_patients_with_f

h_arm1_df_eigen = process_patients_with_f(patients_log[patients_log['arm'] == 1]['PatientFirstName'], nx.eigenvector_centrality_numpy, nx_pathways, mutations_data, True)
h_arm1_df_eigen.describe()

,Cell Cycle,HIPPO,MYC,NOTCH,NRF2,PI3K,RTK-RAS,TGF-Beta,TP53,WNT
count,1.34000000000e+02,1.34000000000e+02,1.34000000000e+02,134.00000000000,134.00000000000,134.00000000000,134.00000000000,134.00000000000,134.00000000000,1.34000000000e+02
mean,8.92723880597e+00,-1.09423105277e-14,-7.45083231823e-12,-0.00000020966,0.32887580074,1.72047317380,0.00003132659,2.03482277523,34.90855181428,1.67838015707e+00
std,6.72526270743e+00,2.58392329967e-13,5.52694308154e-11,0.00000649249,5.28494193823,6.06425581755,0.00007072630,5.63706205896,27.20136770161,8.10376080606e+00
min,-1.44328993201e-15,-2.71063681350e-12,-5.55063524187e-10,-0.00005623898,-20.79034799571,-0.00000000032,-0.00000002807,-0.00008837893,-0.00000000005,-5.62088493125e-10
25%,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000,0.00000000000,0.00000000000,0.00000000067,0.00000000000,0.00000000000,-3.53354944009e-13
50%,1.06250000000e+01,0.00000000000e+00,0.00000000000e+00,0.00000000000,0.00000000000,0.00000000000,0.00000011448,0.00000000000,41.49753421861,0.00000000000e+00
75%,1.37500000000e+01,0.00000000000e+00,0.00000000000e+00,0.00000000000,0.00000000000,0.00000000007,0.00000062075,0.00000000000,54.99603625912,1.07982713507e-12
max,2.10000000000e+01,1.24436720279e-12,1.30662182426e-12,0.00001848903,56.85970521391,37.93195349586,0.00031886862,24.33327569137,83.99762622289,5.79778953608e+01


In [51]:
pandas.set_option("display.precision", 11)
h_arm1_db_df_eigen = h_arm1_df_eigen.join(patients_log.set_index('PatientFirstName'), on='PatientFirstName')
print(f"{h_arm1_db_df_eigen[['dpfs'] + [pw[0] for pw in nx_pathways]].corr().iloc[0]}")

dpfs          1.00000000000
Cell Cycle   -0.12639137193
HIPPO        -0.13213592470
MYC           0.05573437413
NOTCH         0.04797406150
NRF2          0.12443549739
PI3K         -0.00629930667
RTK-RAS       0.05346339497
TGF-Beta     -0.05842360191
TP53         -0.12418430142
WNT          -0.00277919504
Name: dpfs, dtype: float64
